### Input required: Update information

#### Select columns (exclusive: When start is 1 and end is 5, the applicable range will be 2-4.)
Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
start = 'Q1.2'
end = 'Q83'

#### Fill in correct texts for each column.
Note: quotation marks should be included. Change values inside of the quotes.

In [2]:
AY = 'AY22-23'
CL = '2025'
BL = 'MSK'

#### Input a total number of questions/columns per site (for loop purposes). Count all qualitative questions as well.
For example, in this case, we have a total of 43 questions, but will have only 29 quantitiave questions printed in the end.

In [3]:
set_n = 43

That's it. Now you can sit back and run the code! 
#### Initiate

In [4]:
# 1 initiate
from datetime import datetime
import datetime                                               # for timestamp 
import time                                                   # for timers

# start timer
tic = time.time()

import numpy as np
import pandas as pd

#import dataset
readfile = pd.read_excel('input.xlsx', engine='openpyxl')

#create a list of columns
cols = list(readfile.columns)

#subset 16 survey questions
q_start = cols.index(start)
q_end = cols.index(end)
q_subset = readfile.iloc[:,q_start+1:q_end]

#create a list of subset columns
q_cols_subset = cols[q_start+1:q_end]

In [5]:
# 2 create a placeholder dataframe
df0 = pd.DataFrame(columns = ['AcademicYear', 'Class', 'Block'])

#df0

In [6]:
# 2 create a dataframe for faculty name columns

sequence1 = 0
Faculty_FirstName = []
Faculty_LastName = []

# extract from the first question & repeat 17 times
for k in q_cols_subset:
    sequence1 = sequence1 + 1
    if (sequence1%17 == 1):            
        q_text_f = str(q_subset[k][0])



        sub1 = "Dr."
        sub2 = "</"

        # getting index of substrings
        idx1 = q_text_f.index(sub1)
        idx2 = q_text_f.index(sub2)

        res = ''
        # getting elements in between
        for idx in range(idx1 + len(sub1) + 1, idx2):
            res = res + q_text_f[idx]

        # result
        Faculty_FullName = res.split()
        Faculty_FirstName.append(str(Faculty_FullName[0]))
        Faculty_LastName.append(str(Faculty_FullName[1]))

    else:
        Faculty_FirstName.append(str(Faculty_FullName[0]))
        Faculty_LastName.append(str(Faculty_FullName[1]))
        continue

df1 = pd.DataFrame(list(zip(Faculty_FirstName, Faculty_LastName)),
       columns =['Faculty_FirstName', 'Faculty_LastName'])

#df1

In [7]:
# 3 create a dataframe for stats

df2 = pd.DataFrame([])
for j in q_cols_subset:

    #counts & response rates
    c = q_subset[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')
    c['pct'] = c.counts / c.counts.sum()

    cc = {}
    for i in range(1,6):
        if i in list(c.unique_values):
            cn = c[c.unique_values == i]['counts']
            pn = c[c.unique_values == i]['pct']
            cc.update(                {
                    'C'+str(i):int(cn), 'R'+str(i):float(pn)
                }            )
            
        else:
            cc.update(                {
                    'C'+str(i):0, 'R'+str(i):0
                }              )


    dd = pd.DataFrame([cc])
    dd['A_SA'] = dd.R4 + dd.R5
    dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
    dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
    df2 = pd.concat([df2, dd], axis=0, ignore_index=True)
    df2 = df2[['R1', 'R2', 'R3', 'R4', 'R5', 'A_SA', 'Mean', 'N', 'C1', 'C2', 'C3', 'C4', 'C5']]

#df2

In [8]:
# 4 create a dataframe for question
#text cell extract


sequence2=0
q0 = []
for l in q_cols_subset:
    if (sequence2 < 17):        
        sequence2 = sequence2 + 1
        q_text_q = str(q_subset[l][0])  
        q0.append(q_text_q.split("/> - ", 1)[1])
    else:
        break

q1 = pd.DataFrame(q0)

count = 0
df3 = pd.DataFrame([])
while (count < (len(q_cols_subset)/17)):   
    count = count + 1
    df3 = pd.concat([df3, q1], axis=0, ignore_index=True)

df3.columns =['Question']

#df3

In [9]:
# 5 concat three dataframes
df4 = pd.concat([df0, df3, df1, df2], axis=1)

#df4

In [10]:
# 6 delete the 17th qualitative questions
df5 = df4[df4['N'] > 0]

# 7 populate columns for AY, Class, Block
df5 = df5.assign(AcademicYear = AY)
df5 = df5.assign(Class = CL)
df5 = df5.assign(Block = BL)

#df5

In [11]:
#### Export

In [12]:
df5.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

In [13]:
# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 2.16 s.
